This file should contain the data scaling, train/test/split, and model building cells. It should output the fitted model to file.<br>
Obvy it's going to need to read in the data, too, but nothing more complicated than that.<br>

actually hang on<br>

make the whole data preprocessing bit a big ol' function <br>
stuff that in its own file<br>
import it into model building and model evaluation files<br>

then make the actual model building one file<br>
make sure you can do more than 1 callback to a model.fit ... pretty sure I can as long as I pass a list ...<br>
save the model to a file<br>
save the history log to a file<br>

then make sure you call the data preprocessing function in the model evaluation file<br>
read in the history log<br>

In [1]:
# Import analysis packages:
import pandas as pd
import numpy as np

import random

In [2]:
from DataPreprocessing import output_train_test_data

In [3]:
''' I noticed that I was getting several warnings when compiling TensorFlow regarding appropriate
compiler flags; apparently, I'm not the only one, as evidenced by the following question from
stackoverflow:
https://stackoverflow.com/questions/66092421/how-to-rebuild-tensorflow-with-the-compiler-flags
I have chosen to implement the commenter's solution.'''
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, Precision, Recall

In [5]:
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(11)

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(11)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(11)

# Data PreProcessing

In [6]:
scaled_training_data, training_labels, scaled_valid_data, valid_labels, \
     scaled_test_data, test_labels = output_train_test_data()

Proportion of Data Points per Class, Full Data Set:
2    0.4876
1    0.3646
3    0.0615
7    0.0353
6    0.0299
5    0.0163
4    0.0047
Name: class, dtype: float64

Proportion of Data Points per Class, Training Data Subset:
2    0.4876
1    0.3646
3    0.0615
7    0.0353
6    0.0299
5    0.0163
4    0.0047
Name: class, dtype: float64

Proportion of Data Points per Class, Validation Data Subset:
2    0.4876
1    0.3646
3    0.0615
7    0.0353
6    0.0299
5    0.0163
4    0.0047
Name: class, dtype: float64

Proportion of Data Points per Class, Test Data Subset:
2    0.4876
1    0.3646
3    0.0615
7    0.0353
6    0.0299
5    0.0163
4    0.0047
Name: class, dtype: float64
Training Data and Labels Shapes:
(406708, 53) (406708, 7)

Validation Data and Labels Shapes:
(87152, 53) (87152, 7)

Test Data and Labels Shapes:
(87152, 53) (87152, 7)


# Building a model

In [7]:
def build_a_model(xs):
    
    nvars = xs.shape[1]

    thismodel = Sequential()
    thismodel.add(InputLayer(input_shape=(nvars,)))
    thismodel.add(Dense(512, activation='relu'))
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(256, activation='relu'))
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(128, activation='relu'))
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(64, activation='relu'))
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(32, activation='relu'))
    thismodel.add(Dropout(0.001))
    thismodel.add(Dense(16, activation='relu'))
    thismodel.add(Dropout(0.01))
    thismodel.add(Dense(7, activation='softmax'))
    thisoptimizer = Adam(learning_rate=0.005)  
    thismodel.compile(loss='CategoricalCrossentropy',  metrics=['accuracy','Precision','Recall'], \
                      optimizer=thisoptimizer)
    
    return thismodel

In [8]:
earliestop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=40)

# dis from 
# https://stackoverflow.com/questions/47843265/how-can-i-get-a-keras-models-history-after-loading-it-from-a-file-in-python
historylogger = CSVLogger('nn_model_history.log', separator=',', append=False)

In [9]:
nn_model = build_a_model(scaled_training_data)

In [10]:
model_history = nn_model.fit(scaled_training_data, training_labels, \
                             validation_data = (scaled_valid_data,valid_labels), \
                             shuffle = False, epochs = 310, batch_size = 800, \
                             callbacks=[earliestop,historylogger], verbose = 0)

Epoch 00249: early stopping


In [11]:
nn_model.save("cover_model_MAC")

INFO:tensorflow:Assets written to: cover_model_MAC\assets
